In [ ]:
%cd /local/home/ahemf/mygit/facebook-hateful-memes/notebooks/
%pwd
!pip uninstall -y maskrcnn_benchmark
!rm -rf vqa-maskrcnn-benchmark
!git clone https://gitlab.com/vedanuj/vqa-maskrcnn-benchmark.git
# !sed -i '/from maskrcnn_benchmark import _C/c\from ._utils import _C' maskrcnn_benchmark/layers/nms.py
!cat maskrcnn_benchmark/layers/nms.py
%cd vqa-maskrcnn-benchmark

# %sed -i '/from maskrcnn_benchmark import _C/c\from ._utils import _C' layers/nms.py
# %cat layers/nms.py
%pwd
!python setup.py build
!python setup.py develop
!wget -O detectron_model.pth wget https://dl.fbaipublicfiles.com/vilbert-multi-task/detectron_model.pth
!wget -O detectron_model.yaml wget https://dl.fbaipublicfiles.com/vilbert-multi-task/detectron_config.yaml

import sys
import os
DIR = os.getcwd()
sys.path.append(f'{DIR}/vqa-maskrcnn-benchmark')


In [ ]:
# %pip install git+https://github.com/faizanahemad/ImageCaptioning.pytorch.git
%pip install gdown
!gdown --id 1VmUzgu0qlmCMqM1ajoOZxOXP3hiC_qlL
!gdown --id 1zQe00W02veVYq-hdq5WsPOS3OPkNdq79
!pip install yacs



%cd ..
%mkdir model_data
!pip uninstall -y captioning
%pip install git+https://github.com/faizanahemad/ImageCaptioning.pytorch.git


In [ ]:
import pandas as pd
import numpy as np

from PIL import Image
from IPython.display import display, HTML, clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import sys
import os
DIR = os.getcwd()
import torch
import torchvision
from facebook_hateful_memes_detector.utils.globals import set_global, get_global
set_global("cache_dir", "/home/ahemf/cache/cache")
set_global("dataloader_workers", 4)
from facebook_hateful_memes_detector.preprocessing import DefinedRotation, QuadrantCut, DefinedAffine, DefinedColorJitter, DefinedRandomPerspective, ImageAugment
from PIL import Image
from facebook_hateful_memes_detector.utils import get_image_info_fn, set_device, get_device
from torchvision import transforms
import joblib
from tqdm.auto import tqdm, trange
from joblib import Parallel, delayed
from facebook_hateful_memes_detector.preprocessing import TextImageDataset, my_collate, get_datasets, get_image2torchvision_transforms, TextAugment
from facebook_hateful_memes_detector.models.external.detr import get_detr_model

def hash(x):
    return joblib.hashing.hash(x, 'sha1')

def print_code(func):
    import inspect
    from pygments import highlight
    from pygments.lexers import PythonLexer
    from pygments.formatters import TerminalFormatter

    code = "".join(inspect.getsourcelines(func)[0])
    print(highlight(code, PythonLexer(), TerminalFormatter()))

set_device('cuda')



In [ ]:
!wget https://raw.githubusercontent.com/airsplay/py-bottom-up-attention/master/demo/data/images/input.jpg


In [ ]:
data = get_datasets(
    data_dir="../data/",
    train_text_transform=None,
    train_image_transform=None,
    test_text_transform=None,
    test_image_transform=None,
    cache_images=True,
    use_images=True,
    dev=False,
    keep_original_text=False,
    keep_original_image=True,
    keep_processed_image=False,
    keep_torchvision_image=False,
)
augs_dict = dict(
    grayscale=transforms.Grayscale(num_output_channels=3),
    hflip=transforms.RandomHorizontalFlip(p=1.0),
    rc2=transforms.Compose(
        [transforms.Resize(480),
         transforms.CenterCrop(400)]),
    rotate=DefinedRotation(15),
    affine=DefinedAffine(0, scale=(0.6, 0.6)),
    translate1=DefinedAffine(0, translate=(0.25, 0.25)),
#     translate2=DefinedAffine(0, translate=(0.1, 0.1)),
#     color1=DefinedColorJitter(brightness=(0.9, 1.1), contrast=0, saturation=0, hue=0),
#     color2=DefinedColorJitter(contrast=0.1),
#     color3=DefinedColorJitter(saturation=0.1),
#     qcut=QuadrantCut(),
    identity=lambda x: x,
)
im_transform = ImageAugment(count_proba=[0.0, 1.0], augs_dict=augs_dict, choice_probas="uniform")



In [ ]:
img = Image.open("input.jpg")

hashes = Parallel(n_jobs=8, backend='threading')(delayed(lambda i: hash(im_transform(i)))(img.copy()) for i in trange(10000))
hashes = set(hashes)
len(hashes)




In [ ]:
fns = get_image_info_fn(enable_encoder_feats=True, enable_image_captions=False)
cache_fns = [fns["get_img_details"], fns["get_encoder_feats"], 
             fns["get_lxmert_details"], 
             get_detr_model(get_device(), "detr_resnet50")["detr_fn"], 
             get_detr_model(get_device(), "detr_resnet50_panoptic")["detr_fn"], lambda x: x]

images = list(data["train"].img.values) + list(data["test"].img.values)
try:
    from torch.cuda.amp import GradScaler, autocast
    scaler = GradScaler()
    use_autocast = "cuda" in str(get_device())
except:
    pass
for i in tqdm(images):
    img = Image.open(i)
    for k, aug in augs_dict.items():
        img_copy = aug(img.copy())
        for fn in cache_fns:
            with autocast():
                _ = fn(img_copy)
    



# We need Qcut for robustness
# We need DefinedAffine translate since in this problem we care about presence of object not position
# We need Image models trained on classification which only care about presence not position


In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
texts = pd.read_csv("text.csv", header=None)[0].values

m = lambda x: tokenizer.encode_plus(x, add_special_tokens=True, pad_to_max_length=False, truncation=False)
tlens = [len(d['input_ids']) for d in map(m, texts)]

np.percentile(tlens, [97, 99, 99.5, 99.9, 100])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_device(device)

from facebook_hateful_memes_detector.utils import get_image_info_fn



In [ ]:
im_transform(Image.open("input.jpg"))

In [ ]:
torchvision.transforms.RandomPerspective(p=1.0)(Image.open("input.jpg"))

In [ ]:
fns = get_image_info_fn(enable_encoder_feats=True, enable_image_captions=True)


In [ ]:
res, info = fns["feature_extractor"](Image.open("../data/img/08291.png"))
res[:2, :8]
info["boxes"][:4]
info["objects"]


In [ ]:
res, info = fns["get_img_details"](Image.open("../data/img/08291.png"))
res[:2, :8]
info["boxes"][:4]
info["objects"]


In [ ]:
fns["get_image_captions"]("../data/img/08291.png")

In [ ]:
im = im_transform(Image.open("input.jpg"))
im

In [ ]:
instances, roi_features = fns["get_lxmert_details"](im)
instances.pred_boxes.tensor # boxes
roi_features # feats
# (feats, boxes)

In [ ]:
fns["get_encoder_feats"]("../data/img/08291.png")
